In [4]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

In [22]:
# Substitua 'SUA_CHAVE_DE_API_AQUI' pela sua chave de API da NASA
api_key = '9tH08yd4PVyMlQkkRvD21kYFtjVJQgDiubaZsUhs'

def obter_dados_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao acessar a API: {response.status_code}")
        return None

def identifica_inicio_e_fim_mes(datas: str, modo: int):
    datas_split = [d.strip() for d in datas.split(',')]

    # | `modo` | Comportamento esperado                                                             |
    # | ------ | ---------------------------------------------------------------------------------- |
    # | 1      | Retorna **somente o dia** inserido (data início = data fim = data inserida)        |
    # | 2      | Retorna o **mês completo** da data inserida                                        |
    # | 3      | Retorna do **primeiro dia** inserido **até o segundo dia** inserido                |
    # | 4      | Retorna do **início do mês da primeira data** até o **fim do mês da segunda data** |

    
    if modo == 1:
        # Apenas o dia inserido
        if len(datas_split) != 1:
            raise ValueError("Modo 1 espera apenas uma data.")
        data = datetime.strptime(datas_split[0], "%Y-%m-%d")
        return data, data

    elif modo == 2:
        # Mês completo da data inserida
        if len(datas_split) != 1:
            raise ValueError("Modo 2 espera apenas uma data.")
        data = datetime.strptime(datas_split[0], "%Y-%m-%d")
        inicio_mes = data.replace(day=1)
        proximo_mes = (inicio_mes.replace(day=28) + timedelta(days=4)).replace(day=1)
        fim_mes = proximo_mes - timedelta(days=1)
        return inicio_mes, fim_mes

    elif modo == 3:
        # De uma data até a outra (dia específico até outro dia específico)
        if len(datas_split) != 2:
            raise ValueError("Modo 3 espera duas datas.")
        data_inicio = datetime.strptime(datas_split[0], "%Y-%m-%d")
        data_fim = datetime.strptime(datas_split[1], "%Y-%m-%d")
        return data_inicio, data_fim

    elif modo == 4:
        # Do início do mês da primeira data até o fim do mês da segunda data
        if len(datas_split) != 2:
            raise ValueError("Modo 4 espera duas datas.")
        data_inicio = datetime.strptime(datas_split[0], "%Y-%m-%d").replace(day=1)
        data_fim_temp = datetime.strptime(datas_split[1], "%Y-%m-%d")
        proximo_mes = (data_fim_temp.replace(day=28) + timedelta(days=4)).replace(day=1)
        data_fim = proximo_mes - timedelta(days=1)
        return data_inicio, data_fim

    else:
        raise ValueError("Modo inválido. Use 1, 2, 3 ou 4.")


# Função para coletar dados e imagens da API variado por dia
def coletar_e_salvar_dados(intervalo, modo, baixar_imagens):
    data_inicio, data_fim = identifica_inicio_e_fim_mes(intervalo,modo)
    print(data_inicio)
    print(data_fim)

    while data_inicio <= data_fim:
        data_str = data_inicio.strftime('%Y-%m-%d')
        url = f"https://api.nasa.gov/mars-photos/api/v1/rovers/Perseverance/photos?earth_date={data_str}&api_key={api_key}"
        print(f"\n📡 Consultando dados para: {data_str}")
        
        dados = obter_dados_api(url)
        if dados and dados.get('photos'):
            photo_data = []
            for photo in dados['photos']:
                photo_data.append({
                    'photo_id': photo['id'],
                    'sol': photo['sol'],
                    'camera_id': photo['camera']['id'],
                    'camera_name': photo['camera']['name'],
                    'camera_full_name': photo['camera']['full_name'],
                    'img_src': photo['img_src'],
                    'earth_date': photo['earth_date'],
                    'rover_id': photo['rover']['id'],
                    'rover_name': photo['rover']['name'],
                    'rover_landing_date': photo['rover']['landing_date'],
                    'rover_launch_date': photo['rover']['launch_date'],
                    'rover_status': photo['rover']['status'],
                })
            
            df = pd.DataFrame(photo_data)
            
            # Criar diretório da data
            diretorio = f"DadosApiNASA/{data_str}"
            os.makedirs(diretorio, exist_ok=True)
            
            # Salvar CSV com dados
            caminho_csv = os.path.join(diretorio, f"{data_str}_photos.csv")
            df.to_csv(caminho_csv, index=False)
            print(f"📝 CSV salvo em: {caminho_csv}")
            
            # Baixar imagens, se permitido
            if baixar_imagens == 1:
                print("⬇️  Baixando imagens...")
                for photo in photo_data:
                    img_url = photo['img_src']
                    img_path = os.path.join(diretorio, f"{photo['photo_id']}.jpg")
                    response = requests.get(img_url)
                    if response.status_code == 200:
                        with open(img_path, 'wb') as img_file:
                            img_file.write(response.content)
                        print(f"📷 Imagem salva: {img_path}")
                    else:
                        print(f"❌ Erro ao baixar imagem {photo['photo_id']}")
            else:
                print("🚫 Download de imagens desativado.")

        else:
            print(f"⚠️ Nenhuma foto encontrada para {data_str}")
        
        data_inicio += timedelta(days=1)



In [24]:
intervalo = "2025-04-24,2025-04-26"  # ou apenas "2025-04-20"
modo = 3 #modo pode ser 1,2,3 ou 4 a depender do seu preenchimento e desejo de consulta de data
baixar_imagens = 2 #selecione 1 para baixar as imagens e 2 para não baixar as imagens e apenas obter o CSV de cada dia
coletar_e_salvar_dados(intervalo, modo, baixar_imagens)

2025-04-24 00:00:00
2025-04-26 00:00:00

📡 Consultando dados para: 2025-04-24
📝 CSV salvo em: DadosApiNASA/2025-04-24\2025-04-24_photos.csv
🚫 Download de imagens desativado.

📡 Consultando dados para: 2025-04-25
📝 CSV salvo em: DadosApiNASA/2025-04-25\2025-04-25_photos.csv
🚫 Download de imagens desativado.

📡 Consultando dados para: 2025-04-26
⚠️ Nenhuma foto encontrada para 2025-04-26
